# Domain_Filter for MLflow

In [1]:
# !pip install -q requirements.txt

In [ ]:
class S3_provider():
    """
    Класс для взаимодействия с хранилищем S3.

    Этот класс предоставляет методы для загрузки файлов из S3 хранилища. Он используется для загрузки и
    хранения моделей и данных, необходимых для работы сервиса.
    """
    
    def __init__(self):
        """
        Инициализация провайдера S3.

        Настраивает соединение с хранилищем S3, используя заданные параметры подключения.
        """
        # Работа с облачными сервисами
        import s3fs
        import boto3
        from botocore.client import Config
    
        # Настройки MinIO
        minio_access_key  = "minio_access_key"
        minio_secret_key  = "minio_secret_key"
        minio_endpoint    = "minio_endpoint"
        minio_bucket_name = "minio_bucket_name"

        self.s3 = boto3.resource('s3',
                            endpoint_url=minio_endpoint,
                            aws_access_key_id='minio_access_key',
                            aws_secret_access_key='minio_secret_key',
                            config=Config(signature_version='s3v4'),
                            region_name='us-east-1')

        self.bucket_name = 'prod-aiplatform-data'
        self.bucket = self.s3.Bucket(self.bucket_name)

        self.s3 = s3fs.S3FileSystem(anon=False, 
                            key=minio_access_key, 
                            secret=minio_secret_key, 
                            client_kwargs={"endpoint_url": minio_endpoint},
                            use_ssl=False)


    def download_from_s3(self, s3_folder: str, local_folder: str) -> str:
        """
        Загрузка файлов из S3 хранилища в локальную директорию.

        Description:
            Метод автоматически загружает все файлы из указанной папки в S3 хранилище в локальную директорию.
            Если локальная директория не существует, она будет создана вместе с необходимыми поддиректориями.
            Процесс загрузки логируется, предоставляя информацию о статусе загрузки каждого файла.
            В случае возникновения ошибки в процессе загрузки, метод логирует ошибку и возвращает `None`.
        Args:
            s3_folder (str): Путь к папке в S3 хранилище. Указывается от корня бакета.
            local_folder (str): Путь к локальной папке для сохранения файлов.
        Returns:
            str or None: Возвращает путь к локальной директории, куда были загружены файлы, если процесс завершился
                         успешно. Возвращает `None`, если в процессе загрузки произошла ошибка.
        Exceptions:
            Логирует исключения, связанные с ошибками доступа к S3 или невозможностью создать локальные директории.
        """
        import os
        import logging
        
        logging.basicConfig(
            level=logging.INFO,
            format="%(asctime)s - [%(levelname)s]: %(message)s",
            handlers=[
                logging.handlers.RotatingFileHandler(
                    filename="log.log",
                    mode="a",
                    maxBytes=1024,
                    backupCount=1,
                    encoding=None,
                    delay=0),
                logging.StreamHandler()
                ]
              )

        if not os.path.exists(local_folder):
            try:
                for obj in self.bucket.objects.filter(Prefix=s3_folder):
                    # Формирование пути для сохранения файла локально
                    local_path = os.path.join(local_folder, os.path.basename(obj.key))

                    # Создание локальной папки, если она не существует
                    os.makedirs(os.path.dirname(local_path), exist_ok=True)

                    # Загрузка файла из S3 в локальную папку
                    self.bucket.download_file(obj.key, local_path)

                logging.info(f"Файлы успешно загружены из S3 в {local_path}")
            except Exception as e:
                logging.info(f"Ошибка при загрузке файла из S3: {str(e)}")
                return None
        return local_folder

In [2]:
import mlflow
import boto3

class Domain_Filter(mlflow.pyfunc.PythonModel):
    """
    Класс NLU классификатора для определения доменной области сообщения.

    Использует предобученную модель NLP (NLU) для классификации запросов и включает предварительную обработку текста.

    Args:
        tokenizer: Токенизатор для предобученной модели NLU.
        model: Предобученная NLU модель.
        lemmatizer: Лемматизатор для приведения слов к их базовой форме.
    """
    import  pandas as pd
    import torch
    
    def __init__(self, threshold=0.4):
        """
        Инициализирует NLU_Classifier с заданной моделью NLU.

        Args:
            nlu_model (str): Путь к предобученной модели NLU.
            threshold (float, optional): Пороговое значение для сходства между запросом и доменами.
        """

        self.information_security = ['ПАК ЗВП', 'PTAF', 'WAF', 'ПТАФ', 'ВАФ', 'программно-аппаратный комплекс защиты веб приложений', 'веб-файервол', 'web firewall',
                                     'Антивирус', 'KSC', 'Касперский', 'Kaspersky', 'антивирус касперского', 'антивирусная защита', 'АВЗ', 'комплексная система антивирусной защиты',
                                     'КСАЗ', 'NGate', 'шифрование по ГОСТ', 'ГОСТ шифрование', 'НГейт', 'TLS-ГОСТ', 'Hashicorp Vault', 'Vault', 'Вольт', 'Хашикорп вольт',
                                     'менеджер секретов', 'система хранения секретов', 'система управления секретами', 'КриптоПро', 'CryptoPro', 'CSP', 'криптопровайдер',
                                     'СТП', 'ТП', 'техподдержка', 'техническая поддержка', 'служба технической поддержки', 'Центр обеспечения безопасности', 'Центр обеспечения информационной безопасности',
                                     'Security Operations Center', 'SOC', 'СОК', 'NTP', 'сервис точного времени', 'сервис времени', 'Network Time Protocol', 'СЗИ',
                                     'система защиты информации', 'СрЗИ', 'средство защиты информации', 'СКЗИ', 'средство криптозащиты', 'средство криптографической защиты',
                                     'ЭП', 'электронная подпись']

        # NLU модель сравнивает совпадение слов в тексте запроса с эмбендингами по косинусному расстоянию, на основе чего определяется принадлежность к определенной тематике.
        self.embeddings_topics = [
            ('информационная безопасность', self.information_security)
        ]
        
        self.threshold = threshold

    def load_context(self, context) -> None:
        """Загрузка моделей из S3 хранилища """
        from transformers import AutoTokenizer, AutoModel
        
        # Создания экземпляра для взаимодействия с хранилищем S3
        self.s3_provider = S3_provider()
        
        # Загрузка модели
        self.nlu_model = self.s3_provider.download_from_s3(s3_folder='prod/sber_large_mt_nlu_ru', local_folder='sber_large_mt_nlu_ru')

        print('Loading NLU classifierr model...')
        # Загрузка токенизатора для предварительно обученной модели NLU
        self.tokenizer = AutoTokenizer.from_pretrained(self.nlu_model)

        # Загрузка предварительно обученной модели NLU с переносом на GPU для ускорения обработки
        self.model = AutoModel.from_pretrained(self.nlu_model).to(device='cuda')
        print('NLU classifierr model loaded')
        

    def preprocess_text(self, text: str) -> str:
        """
        Description:
            Метод для предобработки текстового запроса.
            Включает удаление специальных символов, нормализацию текста и лемматизацию.
        Args:
            text (str): Входящий текстовый запрос.
        Returns:
            str: Обработанный текстовый запрос.
        """
        import re
        
        # Удаление нежелательных символов, сохранение букв, цифр и основных знаков препинания
        text = re.sub(r'[^\w\s.,!?;:]', '', text)

        # Приведение к нижнему регистру
        text = text.lower()

        # Возвращаем обработанный текст
        return text

    def get_domain_embeddings(self, domain_topic=None) -> list:
        """
        Извлекает векторные представления для заданных доменов.
        
        Description:
            Эта функция обрабатывает темы, связанные с каждым доменом, используя предварительно обученную модель NLP для получения векторных представлений.
            Если указан конкретный домен, обрабатываются только темы этого домена. В противном случае обрабатываются темы всех доменов.
        Args:
            domain_topic (str, optional): Название домена, для которого нужно извлечь векторные представления. Если None, обрабатываются все домены.
        Returns:
            List[Tuple[str, ndarray]]: Список кортежей, содержащих название домена и соответствующее ему векторное представление.
        """
        import torch
        
        # Инициализация списка для хранения векторных представлений доменов
        domain_embeddings = []

        if domain_topic:
            # Если указан конкретный домен, обрабатываем только его темы
            for domain, topics in self.embeddings_topics:
                if domain == domain_topic:
                    # Обработка каждой темы в указанном домене
                    for topic in topics:
                        # Предобработка текста темы
                        processed_text = self.preprocess_text(topic)
                        # Токенизация обработанного текста
                        inputs = self.tokenizer(processed_text, return_tensors="pt").to('cuda')

                        # Получение модельного вывода без обратного распространения ошибки
                        with torch.no_grad():
                            model_output = self.model(**inputs)

                        # Вычисление среднего пулинга для получения векторного представления
                        domain_embedding = self.mean_pooling(model_output, inputs['attention_mask'])
                        # Перевод векторного представления в numpy массив
                        domain_embedding = domain_embedding.cpu().numpy()

                        # Добавление векторного представления домена в список
                        domain_embeddings.append((domain, domain_embedding))
                    # Прерывание цикла после обработки указанного домена
                    break
        else:
            # Если домен не указан, обрабатываем все домены
            for domain, topics in self.embeddings_topics:
                # Обработка каждой темы в каждом домене
                for topic in topics:
                    # Аналогичная обработка, как описано выше
                    processed_text = self.preprocess_text(topic)
                    inputs = self.tokenizer(processed_text, return_tensors="pt").to('cuda')

                    with torch.no_grad():
                        model_output = self.model(**inputs)

                    domain_embedding = self.mean_pooling(model_output, inputs['attention_mask'])
                    domain_embedding = domain_embedding.cpu().numpy()

                    domain_embeddings.append((domain, domain_embedding))

        # Возвращение списка векторных представлений
        return domain_embeddings


    def mean_pooling(self, model_output, attention_mask: torch.Tensor) -> torch.Tensor:
        """
        Выполняет усреднение пулинга для токенов.

        Эта функция используется для агрегирования выходных данных модели (представлений токенов)
        в одно усредненное векторное представление для каждого входного примера.

        Параметры:
            model_output: Выходные данные модели.
            attention_mask: Маска внимания.

        Возвращает:
            torch.Tensor: Усредненное векторное представление.
        """
        import torch
        
        # Получение векторных представлений токенов из последнего скрытого состояния модели
        token_embeddings = model_output.last_hidden_state

        # Расширение маски внимания для соответствия размерам токенных векторов
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.shape)

        # Умножение каждого токенного вектора на его маску внимания и суммирование
        # для получения общего векторного представления для каждого примера
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)

        # Подсчет количества токенов для каждого примера (с учетом маски внимания)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)

        # Деление суммы векторов на количество токенов для получения усредненного представления
        return sum_embeddings / sum_mask

    def predict(self, context, model_input: pd.DataFrame) -> pd.DataFrame:
        """
        Предсказывает, соответствует ли запрос заданным доменам.

        Функция сравнивает векторное представление запроса с векторными представлениями доменов.
        Если сходство выше заданного порога, то возвращает соответствующий домен или булево значение, показывающее, превышает ли сходство порог.

        Args:
            model_input (pd.DataFrame): DataFrame содержащий query и domain_topics
            query (str): Текст запроса для анализа.
            domain_topics (List[str], optional): Список доменов для сравнения с запросом. Если None, используются все домены.

        Returns:
            Union[str, bool, None]: Название домена, наиболее соответствующего запросу, если domain_topics=None;
            В противном случае возвращает True или False в зависимости от того, превышает ли сходство порог.
        """
        import torch
        
        # Предобработка DataFrame
        row = model_input.iloc[0]
        user_id       = row['id']
        query         = row['query']
        domain_topics = row['domain class']
        
        # Предобработка входного запроса
        processed_query = self.preprocess_text(query)

        # # Токенизация обработанного запроса
        inputs = self.tokenizer(processed_query, return_tensors="pt", padding=True, truncation=True, max_length=512)
        
        # Перемещаем тензоры на GPU
        inputs = {k: v.to('cuda') for k, v in inputs.items()}

        # Получение модельного вывода без обратного распространения ошибки
        with torch.no_grad():
            model_output = self.model(**inputs)
        # Вычисление векторного представления запроса
        sentence_embedding = self.mean_pooling(model_output, inputs['attention_mask']).cpu().numpy().reshape(1, -1)
        # Получение векторных представлений для указанных доменов
        domain_embeddings = self.get_domain_embeddings(domain_topics)
        max_similarity = -1
        best_match = None

        # Поиск домена с наибольшим косинусным сходством
        for domain, domain_embedding in domain_embeddings:
            domain_embedding = domain_embedding.reshape(1, -1)
            # Вычисление косинусного сходства между запросом и доменом
            cos_similarity = cosine_similarity(sentence_embedding, domain_embedding)[0][0]

            # Обновление лучшего сходства и домена
            if cos_similarity > max_similarity:
                max_similarity = cos_similarity
                best_match = domain

        # Возвращение результата в зависимости от указанных доменов
        if domain_topics is None:
            # Возвращение лучшего домена или False, если сходство ниже порога
            result = best_match if max_similarity >= self.threshold else False
            res = pd.DataFrame({'user_id': [user_id], 'domain_answer': [result]}).to_json()
            return res
        else:
            # Возвращение булевого значения: True, если сходство выше порога
            result = max_similarity >= self.threshold
            res = pd.DataFrame({'user_id': [user_id], 'domain_answer': [result]}).to_json()
            return res

---
### Local test

In [3]:
import pandas as pd
import requests
import json
import base64

# Создаем DataFrame с указанными полями и текстом
data = {
    "id": [42],
    "query": ["Как защититься от вирусов в интернете?"],
    "domain class": ["информационная безопасность"]
}

input_df = pd.DataFrame(data)

output = pd.DataFrame([True], columns=["prediction"])

In [4]:
# Установка URI для MLflow трекинг сервера
mlflow.set_tracking_uri("http://mlflow")

# # Создание эксперимента
# mlflow.create_experiment('domain_nlu_filter')

In [5]:
# Начало MLflow эксперимента
with mlflow.start_run(experiment_id=19):
        mlflow.pyfunc.log_model(
            artifact_path='domain_filter',
            python_model=Domain_Filter(),
            signature=mlflow.models.signature.infer_signature(input_df, output),
            artifacts={"embeddings_topics": './embeddings_topics.json'},
            registered_model_name='domainfilter')

/opt/conda/lib/python3.11/site-packages/mlflow/models/signature.py:130: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input)
2024/03/15 07:20:49 WARNING mlflow.utils.requirements_utils: Found torch version (2.1.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.1.1' without the local v

In [6]:
import mlflow
logged_model = 'runs:/588d30e769894e9e836b73f1868773e7/domain_filter'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

Loading NLU classifierr model...


/opt/conda/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


NLU classifierr model loaded


In [7]:
answer = json.loads(loaded_model.predict(pd.DataFrame(data)))

In [8]:
# Обработка ответа
result = json.loads(loaded_model.predict(pd.DataFrame(data)))
is_relevant = result['domain_answer']['0']

print(f"Query relevance: {is_relevant}")

Query relevance: True


In [27]:
# Завершение MLflow эксперимента
mlflow.end_run()

### Web test

In [2]:
# Создание словаря с данными
data = {
    'id': "42",
    'query': [base64.b64encode("Как защититься от DDoS атак?".encode("utf-8")).decode("utf-8")],
    "domain class": [base64.b64encode("информационная безопасность".encode("utf-8")).decode("utf-8")]
}

df = pd.DataFrame(data)

# Передаем запрос в Domain retriever для определения контекста
model_url = 'https://your_url'

response = json.loads(requests.post(model_url, json={'dataframe_records': df.to_dict(orient='records')}).json()['domain_answer'])

Query relevance: True
